# Exercise 2.3 Colorbars
prepared by M.Hauser

We got to know colorbars in Exercises 1.3 and 1.5. However, there are some quirks when adding colorbars to map plots. 
We have already seen that colorbars can be too large for map plots, which does not happen for 'normal' plots (with exceptions). In this exercise, we will discuss how to overcome this problem.

> This exercise applies only to map plots (or plots where the aspect ratio must be equal).



## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
import mplotutils as mpu

## Load data

We use a NetCDF with historical, and projected climatological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012).

The data was prepared in [another notebook](../data/prepare_CMIP5_map.ipynb).

In [ ]:
file = "../data/cmip5_delta_pr_rcp85_map.nc"

# load data, omitting some unnecessary variables
pr = xr.open_dataset(file, drop_variables=["agree_sign", "pval"])

## Reminder

As noted before, colorbars are usually created as

```python
plt.colorbar(h, ax=ax)
```
    
### Example

We plot the relative change in precipitation between a historical and future time period, calculated from CMIP5 data - including a colorbar:

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))

h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.pr_rel,
    transform=ccrs.PlateCarree(),
    cmap="BrBG",
    vmin=-50,
    vmax=50,
)

ax.set_title("Precipitation")
ax.coastlines()

# =======
# add colorbar
plt.colorbar(h, ax=ax, label="Relative change (%)")

## The problem

The colorbar is larger than the axes! Depending on the map and the extent of the map, this can be even worse. Let's illustrate this with another projection:

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
# ====

f, ax = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.Orthographic(central_latitude=45))
)

ax.coastlines()
h = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree())

plt.colorbar(h, extend="both", orientation="horizontal")

#### Why is this a problem for map plots but not for others?

We have seen examples where the colorbar just worked fine. The issue is that the aspect ratio of a map plot has to be equal, otherwise the map would be distorted. To achieve this, matplotlib shrinks the axes, but does not shrink the figure itself. The colorbar is correct if we set the aspect to `"auto"`, but then of course the map is distorted...

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
# ====

f, ax = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.Orthographic(central_latitude=45))
)

ax.coastlines()
h = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree())

plt.colorbar(h, extend="both", orientation="horizontal")

ax.set_aspect("auto")

# The solution

There is a function in mplotutils that creates colorbars of the correct size: `mpu.colorbar(...)`.

The solution is inspired by this [stackoverflow answer](https://stackoverflow.com/a/30077745). The trick is to read out the coordinates of the cartopy axes and adjust the position of the colorbar accordingly. Because the position of the cartopy axes can change, we have to redo this every time the plot gets drawn.

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
# ====

f, ax = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.Orthographic(central_latitude=45))
)

ax.coastlines()
h = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree())

# =======
# NOTE: mpu.colorbar instead of plt.colorbar
cbar = mpu.colorbar(h, ax, extend="both", orientation="horizontal", aspect=15, pad=0.05)

cbar.set_ticks(np.arange(-1, 1.1, 0.5))

Have a look at the help:

In [ ]:
# uncomment to get the docstring
# mpu.colorbar?

### Exercise

Let's go back to the relative precipitation change

 * Replace the `plt` colorbar with the `mpu` one.
 
 > Note: we use xarray to plot the data - therefore we need to set `add_colorbar=False`

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = pr.hist.plot(
    transform=ccrs.PlateCarree(), cmap="Blues", vmin=0, vmax=4000, add_colorbar=False
)

# =======
# use mpu.colorbar instead
cbar = plt.colorbar(h, label="(mm)")

### Solution

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

h = pr.hist.plot(
    transform=ccrs.PlateCarree(), cmap="Blues", vmin=0, vmax=4000, add_colorbar=False
)

# =======
# use mpu.colorbar instead
cbar = mpu.colorbar(h, ax, label="(mm)")

## More than one axes

You can add a colorbar that spans more than one axes. To achieve that you need to pass a list (or array) of axes to `mpu.colorbar`.

``` ipython
cbar = mpu.colorbar(h, axs)
```


### Exercise

Here we plot the precipitation climatology of the historical and future period.

 * Add a `'horizontal'` colorbar that spans both axes in the plot showing historical and projected rainfall amounts
 * The colorbar is a bit large, play with the `aspect` keyword to find a better size.
 * Save the figure as a pdf
 * Is the colorbar at the right position in the saved figure?


In [ ]:
f, axs = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axs[0]
h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.hist,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    rasterized=True,
)

ax.set_title("Precipitation: historical")

ax = axs[1]
h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.proj,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    rasterized=True,
)

ax.set_title("Precipitation: projections")


for ax in axs:
    ax.coastlines()

# =======
# create the colorbar
# cbar = mpu.colorbar(...)

# save figure
# plt.savefig("pr_hist_proj.pdf")

### Solution


In [ ]:
f, axs = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axs[0]
h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.hist,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    rasterized=True,
)

ax.set_title("Precipitation: historical")

ax = axs[1]
h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.proj,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    rasterized=True,
)

ax.set_title("Precipitation: projections")

for ax in axs:
    ax.coastlines()


# create the colorbar
cbar = mpu.colorbar(h, axs, orientation="horizontal", aspect=30)
cbar.set_label("(mm)")

# save figure
plt.savefig("pr_hist_proj.pdf")

## Some more stuff you can do

* Span multiple axes
* More than one colorbar per axes



In [ ]:
f, axs = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.PlateCarree()))

axs = axs.flatten()

for ax in axs:
    ax.coastlines(color="0.3", lw=0.5)
    ax.set_global()

# plot dummy data to get the colormap
h0 = ax.pcolormesh([[0, 1]])
h1 = ax.pcolormesh([[0, 1]])

h2_1 = ax.pcolormesh([[0, 1]], cmap="Blues")
h2_2 = ax.pcolormesh([[0, 1]], cmap="Reds_r")


h3 = ax.pcolormesh([[0, 1]], cmap="BrBG")

# ====
# single colorbar
cbar = mpu.colorbar(h1, axs[0:2], size=0.2, pad=0.1, orientation="horizontal")

# ====
# two colorbars for the same axes
cbar = mpu.colorbar(h2_1, axs[2], size=0.2, pad=0.1, orientation="horizontal")
cbar.ax.set_xticklabels([])

cbar = mpu.colorbar(h2_2, axs[2], size=0.2, pad=0.4, orientation="horizontal")
cbar.ax.set_xticklabels([])

# ====
# colorbar for three axes
cbar = mpu.colorbar(h3, axs[-3:], size=0.2, pad=0.1, orientation="horizontal")

## shrink and shift

You can use the `shrink` and `shift` keywords to adjust the position of the colorbar. `shrink` and `shift` are in fraction of the total width/ height of the colorbar.


In [ ]:
f, axs = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.Robinson()))

axs = axs.flatten()

# plot dummy data to get a colormap
h = ax.pcolormesh([[0, 1]])

ax = axs[0]
mpu.colorbar(h, ax, orientation="horizontal", shrink=0.5)
ax.text(0.5, 0.5, "shrink=0.5\nshift='symmetric' (default)", ha="center", va="center")

ax = axs[1]
mpu.colorbar(h, ax, orientation="horizontal", shrink=0.5, shift=0.35)
ax.text(0.5, 0.5, "shrink=0.5\nshift=0.35", ha="center", va="center")

ax = axs[2]
mpu.colorbar(h, ax, orientation="horizontal", shrink=0.5, shift=0)
ax.text(0.5, 0.5, "shrink=0.5\nshift=0.", ha="center", va="center")

ax = axs[3]
mpu.colorbar(h, ax, orientation="horizontal", shrink=None, shift=0.2)
ax.text(0.5, 0.5, "shrink=None (default)\nshift=0.2", ha="center", va="center")

### Exercise

Now we plot the historical and future precipitation climatologies and their relative change.

 * Add one colorbar for the climatologies and one for the relative change
 * Add the units with `cbar.ax.set_xlabel`
 * Use `shrink` and `shift` to make room for the xlabel
 > You may have to use different values for `shrink` and `shift` for the two colorbars
> 
 > Use `labelpad` for `set_xlabel` to have some distance between the colorbar and the label
 * Set `extend='both'` and `extendfrac=0.1` for the second colorbar
 > Per default `mpu.colorbar` sets the width of the colorbar with its aspect ratio. As the two are not of the same height, it is recommended to use `size` instead. 

In [ ]:
f, axs = plt.subplots(3, 1, subplot_kw=dict(projection=ccrs.Robinson()))

f.set_size_inches(w=10 / 2.54, h=16 / 2.54)

ax = axs[0]
h0 = pr.hist.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    add_colorbar=False,
)


ax = axs[1]
h1 = pr.proj.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    add_colorbar=False,
)


ax = axs[2]
h2 = pr.pr_rel.plot(
    transform=ccrs.PlateCarree(),
    cmap="BrBG",
    vmin=-50,
    vmax=50,
    add_colorbar=False,
)


axs[0].set_title("Precipitation: historical")
axs[1].set_title("Precipitation: projections")
axs[2].set_title("Precipitation: change")

for ax in axs:
    ax.coastlines()

# first colorbar


# second colorbar



### Solution

As shift is relative to the height of the colorbar, I use `shift=0.1` for the upper colorbar and `shift=0.2` for the lower colorbar. This creates approximately the same absolute shift.

In [ ]:
f, axs = plt.subplots(3, 1, subplot_kw=dict(projection=ccrs.Robinson()))

f.set_size_inches(w=10 / 2.54, h=16 / 2.54)

ax = axs[0]
h0 = pr.hist.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    add_colorbar=False,
)


ax = axs[1]
h1 = pr.proj.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
    add_colorbar=False,
)


ax = axs[2]
h2 = pr.pr_rel.plot(
    transform=ccrs.PlateCarree(),
    cmap="BrBG",
    vmin=-50,
    vmax=50,
    add_colorbar=False,
)


axs[0].set_title("Precipitation: historical")
axs[1].set_title("Precipitation: projections")
axs[2].set_title("Precipitation: change")

for ax in axs:
    ax.coastlines()


# first colorbar
cbar = mpu.colorbar(h1, axs[0:2], shift=0.075, size=0.05)
cbar.ax.set_xlabel("(mm)", labelpad=5, ha="left")


# second colorbar
cbar = mpu.colorbar(h2, axs[2], shift=0.15, size=0.05, extendfrac=0.1, extend="both")
cbar.ax.set_xlabel("(%)", labelpad=10, ha="left")

## Bonus: Alternative Solution

There is a second way to create a colorbar that has the right size - using `axes_grid1`.

"[axes_grid1](https://matplotlib.org/2.0.2/mpl_toolkits/axes_grid/users/overview.html) is a collection of helper classes to ease displaying (multiple) images with matplotlib. In matplotlib, the axes location (and size) is specified in the normalized figure coordinates, which may not be ideal for displaying images that needs to have a given aspect ratio."

 > However, it is not part of the core matplotlib functionality, and not its best-documented part

From the axes_grid1 toolkit we need `make_axes_locatable`:

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
# ====

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Orthographic(central_latitude=45)))
ax.coastlines()

h = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree())

# =======
# add colorbar

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes("bottom", size="6.5%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(h, cax=cbax, orientation="horizontal", extend="both")


> You need to pass `axes_class=plt.Axes` to `append_axes`, else it fails miserably (because it tries to create a new axes with a projection). 


### Exercise

 * Add a vertical colorbar to the historical precipitation plot

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.LambertAzimuthalEqualArea()))
ax.coastlines()

h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.hist,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
)

# add colorbar here

### Solution

In [ ]:
f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.LambertAzimuthalEqualArea()))
ax.coastlines()

h = ax.pcolormesh(
    pr.lon,
    pr.lat,
    pr.hist,
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    vmin=0,
    vmax=4000,
)

# add colorbar here

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes("right", size="6.5%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(h, cax=cbax, orientation="vertical", extend="max")

### Limitations

The solution with `axes_grid1` works reasonably well, but it has limitations, e.g. as far as I know you cannot shrink the colorbar to make room for a label below it.